In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

In [2]:
# Read the input data set
io_folder = 'C:/UCI/Predictive_Modelling/Data/{}'

input_data = pd.read_csv(io_folder.format('Zambia.csv')) # Replace Zambia.csv with Raleigh.csv, if required
input_data

,Name,Mode_final,Length,Ntranscripts,ENC,TauIndex,TissueSpecific,Expression,Chr2,Recombination,Secretion,ImmResp,Protease,SpermComp2
0,FBgn0262006,Relaxed,464,1,68.527,0.998904,1,AG,A,1.741026,nottransfered,N,N,N
1,FBgn0036154,Relaxed,2078,2,61.876,0.998979,1,AG,A,3.045938,nottransfered,Y,N,N
2,FBgn0036158,Relaxed,782,1,55.426,0.999364,1,AG,A,3.184389,nottransfered,N,N,N
3,FBgn0031468,Relaxed,2515,1,51.827,0.980826,1,AG,A,2.228226,transfered,N,N,N
4,FBgn0015583,Relaxed,767,1,57.888,0.999111,1,AG,A,3.283702,transfered,N,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,FBgn0259975,Constrained,396,1,56.411,0.991108,1,other,A,1.305770,transfered,N,N,Y
250,FBgn0259968,Constrained,451,2,39.309,0.990510,1,other,A,0.000000,transfered,N,N,Y
251,FBgn0038014,Constrained,1289,1,60.105,0.999002,1,AG,A,1.741026,transfered,N,Y,N
252,FBgn0083966,Constrained,974,1,59.916,0.999227,1,AG,A,1.196956,transfered,N,Y,Y


In [4]:
# Identify the variables to be used for model development
continuous_var = ['Length', 'Ntranscripts', 'TauIndex', 'Recombination']

In [6]:
# Obtain the sub-matrix for model building and testing
sub_matrix = input_data[continuous_var].copy()
sub_matrix

,Length,Ntranscripts,TauIndex,Recombination
0,464,1,0.998904,1.741026
1,2078,2,0.998979,3.045938
2,782,1,0.999364,3.184389
3,2515,1,0.980826,2.228226
4,767,1,0.999111,3.283702
...,...,...,...,...
249,396,1,0.991108,1.305770
250,451,2,0.990510,0.000000
251,1289,1,0.999002,1.741026
252,974,1,0.999227,1.196956


In [7]:
# Obtain the number of input variables
num_var = sub_matrix.shape[1]
num_var

4

In [8]:
# Number of observations per class
np.unique(input_data['Mode_final'], return_counts=True)

(array(['Constrained', 'Positive', 'Relaxed'], dtype=object),
 array([ 99,  31, 124], dtype=int64))

In [9]:
# Map class labels to indexes
response_variable = input_data['Mode_final'].replace(to_replace = ['Constrained', 'Positive', 'Relaxed']
                                                     , value = [0, 1, 2])

In [10]:
# Obtain the weights per class as a Tensor
# weight_per_class = torch.tensor([1., 3., 0.8])
weight_per_class = torch.tensor([1., 1., 1])

In [11]:
# Check the splitting of data into batches
# num_batch = np.ceil(X_train_scaled.shape[0]/32).astype(int)
num_batch = 1

# grp_indexes = np.random.randint(num_batch, size=X_train_scaled.shape[0])
# np.unique(grp_indexes, return_counts=True)

### Model 1: Logistic Regression 

In [12]:
class Logistic_Regression(nn.Module):
    def __init__(self, num_inputs):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = torch.nn.Linear(num_inputs, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        return x

In [13]:
nnet_model = Logistic_Regression(num_var)
nnet_loss_func = nn.CrossEntropyLoss(weight = weight_per_class) #All classes are treated approximately equally
nnet_optimizer = torch.optim.Adam(nnet_model.parameters(), lr=0.1)

In [14]:
# Number of model parameters
num_param = 0

for param_itr in nnet_model.parameters():
    num_param += param_itr.numel()
    
num_param    

15

### Model 2: Neural Network with two hidden layers

In [15]:
class Hidden_Layer_Two(nn.Module):
    def __init__(self, num_inputs):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = torch.nn.Linear(num_inputs, 3)
        self.fc2 = torch.nn.Linear(3, 3)
        self.fc3 = torch.nn.Linear(3, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x

In [16]:
hnet_model = Hidden_Layer_Two(num_var)
hnet_loss_func = nn.CrossEntropyLoss(weight = weight_per_class) #All classes are treated approximately equally
hnet_optimizer = torch.optim.Adam(hnet_model.parameters(), lr=0.1)

In [17]:
# Number of model parameters
num_param = 0

for param_itr in hnet_model.parameters():
    num_param += param_itr.numel()
    
num_param  

39

### Final Results
Based on 100 iterations using Logistic Regression

In [18]:
rng = np.random.default_rng(100)

per_epoch_loss = []
test_accuracy_list = []
type_0_accuracy = []
type_1_accuracy = []
type_2_accuracy = []

for itr_count in range(100):

    if itr_count%10 ==0:
        print('Iteration = ', itr_count)
    
    # Generate test-train split
    split_seed = rng.integers(2**30)
    X_train, X_test, y_train, y_test = train_test_split(sub_matrix, response_variable, test_size=0.33
                                                        , random_state=split_seed, stratify=response_variable)

    # Use ONLY training set to estimate the parameters for data normalization
    scaler = StandardScaler()
    scaler.fit(X_train[continuous_var])

    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[continuous_var] = scaler.transform(X_train[continuous_var])
    X_test_scaled[continuous_var] = scaler.transform(X_test[continuous_var])
    grp_indexes = np.random.randint(num_batch, size=X_train_scaled.shape[0])

    # Convert training and test data sets to tensors
    X_train_scaled = torch.tensor(X_train_scaled.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values)

    X_test_scaled = torch.tensor(X_test_scaled.values, dtype=torch.float32)

    # Perform logistic regression
    nnet_model = Logistic_Regression(num_var)
    nnet_loss_func = nn.CrossEntropyLoss(weight = weight_per_class) # Up- or down-sample observations to reduce class imbalance
    nnet_optimizer = torch.optim.Adam(nnet_model.parameters(), lr=0.1)

    num_epochs = 10

    for i in range(num_epochs):
        
        for batch_idx in range(num_batch):
            
            current_batch_idx = np.where(grp_indexes == batch_idx)[0].tolist()
            current_fit = nnet_model(X_train_scaled[current_batch_idx, :])  # 1
            current_loss = nnet_loss_func(current_fit, y_train[current_batch_idx])  # 2
            nnet_optimizer.zero_grad()  # 3
            current_loss.backward()  # 4
            nnet_optimizer.step()  # 5
            per_epoch_loss.append(current_loss.detach().numpy().tolist())
    
    test_predictions = torch.argmax(torch.softmax(nnet_model(X_test_scaled), 1), axis=1).numpy()
    test_accuracy = float(sum(test_predictions == y_test))/y_test.shape[0]
    test_accuracy_list.append(test_accuracy)
        
    # Obtain the accuracy per class
    zero_idx = np.where(y_test == 0)[0] 
    one_idx = np.where(y_test == 1)[0] 
    two_idx = np.where(y_test == 2)[0]
    
    type_0_accuracy.append(sum(test_predictions[zero_idx] == 0)/len(zero_idx))
    type_1_accuracy.append(sum(test_predictions[one_idx] == 1)/len(one_idx))
    type_2_accuracy.append(sum(test_predictions[two_idx] == 2)/len(two_idx))

Iteration =  0
Iteration =  10
Iteration =  20
Iteration =  30
Iteration =  40
Iteration =  50
Iteration =  60
Iteration =  70
Iteration =  80
Iteration =  90


In [19]:
len(test_accuracy_list)

100

In [20]:
# Save the results for plotting using ggplot
# accuracy_frame = pd.DataFrame(test_accuracy_list)
# accuracy_frame.to_csv('Overall_Accuracy_Zambia.csv', index=False, header=False)

# accuracy_frame = pd.DataFrame(type_0_accuracy)
# accuracy_frame.to_csv('Type_0_Accuracy_Zambia.csv', index=False, header=False)

# accuracy_frame = pd.DataFrame(type_1_accuracy)
# accuracy_frame.to_csv('Type_1_Accuracy_Zambia.csv', index=False, header=False)

# accuracy_frame = pd.DataFrame(type_2_accuracy)
# accuracy_frame.to_csv('Type_2_Accuracy_Zambia.csv', index=False, header=False)

In [21]:
# Obtain the 5-figure summary
np.percentile(test_accuracy_list, [0, 25, 50, 75, 100])

array([0.47619048, 0.57142857, 0.60119048, 0.63095238, 0.66666667])

In [22]:
np.mean(test_accuracy_list) * 100

59.63095238095239

In [23]:
# Acuracy per class - Class 0
print(np.percentile(type_0_accuracy, [0, 25, 50, 75, 100]))

np.mean(type_0_accuracy) * 100

[0.3030303  0.42424242 0.48484848 0.51515152 0.75757576]


47.75757575757575

In [24]:
# Acuracy per class - Class 1
print(np.percentile(type_1_accuracy, [0, 25, 50, 75, 100]))

np.mean(type_1_accuracy) * 100

[0.  0.  0.  0.  0.2]


0.9000000000000001

In [25]:
# Acuracy per class - Class 2
print(np.percentile(type_2_accuracy, [0, 25, 50, 75, 100]))

np.mean(type_2_accuracy) * 100

[0.63414634 0.75609756 0.85365854 0.90243902 1.        ]


83.51219512195121

Based on 100 iterations of a 2-hidden layer model

In [26]:
rng = np.random.default_rng(100)

per_epoch_loss = []
test_accuracy_list = []
type_0_accuracy = []
type_1_accuracy = []
type_2_accuracy = []

for itr_count in range(100):

    if itr_count%10 ==0:
        print('Iteration = ', itr_count)
    
    # Generate test-train split
    split_seed = rng.integers(2**30)
    X_train, X_test, y_train, y_test = train_test_split(sub_matrix, response_variable, test_size=0.33
                                                        , random_state=split_seed, stratify=response_variable)

    # Use ONLY training set to estimate the parameters for data normalization
    scaler = StandardScaler()
    scaler.fit(X_train[continuous_var])

    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[continuous_var] = scaler.transform(X_train[continuous_var])
    X_test_scaled[continuous_var] = scaler.transform(X_test[continuous_var])
    grp_indexes = np.random.randint(num_batch, size=X_train_scaled.shape[0])

    # Convert training and test data sets to tensors
    X_train_scaled = torch.tensor(X_train_scaled.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values)

    X_test_scaled = torch.tensor(X_test_scaled.values, dtype=torch.float32)

    # Perform logistic regression
    nnet_model = Hidden_Layer_Two(num_var)
    nnet_loss_func = nn.CrossEntropyLoss(weight = weight_per_class) # Up- or down-sample observations to reduce class imbalance
    nnet_optimizer = torch.optim.Adam(nnet_model.parameters(), lr=0.1)

    num_epochs = 10

    for i in range(num_epochs):
        
        for batch_idx in range(num_batch):
            
            current_batch_idx = np.where(grp_indexes == batch_idx)[0].tolist()
            current_fit = nnet_model(X_train_scaled[current_batch_idx, :])  # 1
            current_loss = nnet_loss_func(current_fit, y_train[current_batch_idx])  # 2
            nnet_optimizer.zero_grad()  # 3
            current_loss.backward()  # 4
            nnet_optimizer.step()  # 5
            per_epoch_loss.append(current_loss.detach().numpy().tolist())
    
    test_predictions = torch.argmax(torch.softmax(nnet_model(X_test_scaled), 1), axis=1).numpy()
    test_accuracy = float(sum(test_predictions == y_test))/y_test.shape[0]
    test_accuracy_list.append(test_accuracy)
        
np.mean(test_accuracy_list)        

Iteration =  0
Iteration =  10
Iteration =  20
Iteration =  30
Iteration =  40
Iteration =  50
Iteration =  60
Iteration =  70
Iteration =  80
Iteration =  90


0.6173809523809524

### Model 3: XGBoost

In [27]:
rng = np.random.default_rng(100)

test_accuracy_list = []
type_0_accuracy = []
type_1_accuracy = []
type_2_accuracy = []

# Set the parameters
param = {'max_depth': 6, 'eta': 0.3, 'objective': 'multi:softmax', 'num_class': 3, 'eval_metric': 'mlogloss'}

for itr_count in range(100):

    if itr_count%10 ==0:
        print('Iteration = ', itr_count)
    
    # Generate test-train split
    split_seed = rng.integers(2**30)
    X_train, X_test, y_train, y_test = train_test_split(sub_matrix, response_variable, test_size=0.33
                                                        , random_state=split_seed, stratify=response_variable)

    # Use ONLY training set to estimate the parameters for data normalization
    scaler = StandardScaler()
    scaler.fit(X_train[continuous_var])

    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[continuous_var] = scaler.transform(X_train[continuous_var])
    X_test_scaled[continuous_var] = scaler.transform(X_test[continuous_var])

    dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
    dtest = xgb.DMatrix(X_test_scaled)
    bst = xgb.train(param, dtrain, 10)
    test_predictions = bst.predict(dtest)
    
    test_accuracy = float(sum(test_predictions == y_test))/y_test.shape[0]
    test_accuracy_list.append(test_accuracy)
        
np.mean(test_accuracy_list)        

Iteration =  0
Iteration =  10
Iteration =  20
Iteration =  30
Iteration =  40
Iteration =  50
Iteration =  60
Iteration =  70
Iteration =  80
Iteration =  90


0.566547619047619